In diesem Szenario02 werden drei Fußgänger auf die Straße gespawnt. Sie überqueren die Straße von rechts nach links.
Die einzelnen Fußgänger sind auf unterschiedlichen Positionen und besitzten unterschiedliche Geschwindigkeiten.
Das Auto ist auf Autopilot gestellt. 

In [ ]:
%pip install carla

In [ ]:
import carla
import math
import random
import time
import numpy as np
import cv2
import open3d as o3d 
from matplotlib import cm 

In [ ]:
# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

#Load a World
client.load_world('Town05')


In [ ]:
#Add the ego_vehicle
vehicle_bp_benz= bp_lib.find('vehicle.mercedes.coupe_2020')
vehicle_bp_benz.set_attribute('color', '255,255,255')
ego_vehicle = world.spawn_actor(vehicle_bp_benz, spawn_points[2])

In [ ]:
#Move spectator behind to view
spectator = world.get_spectator()
transform = carla.Transform(ego_vehicle.get_transform().transform(carla.Location(x=-6, z=2.0)), ego_vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [ ]:
#Iterate this cell to find desired camera location
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2)) #Change this to move camera
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

#time.sleep(0.2)
#camera.destroy()

In [ ]:
#Mit KI, nicht so nice

def spawn_pedestrian_freePosition(self, distance_x, distance_y):
    transform = carla.Transform(self.get_transform().transform(carla.Location(x = distance_x, y = distance_y, z = 1)))
    spawn_pedestrian(transform)

def spawn_pedestrian(transform):
    bp_walker = world.get_blueprint_library().filter('walker.pedestrian.*')
    bp_walker_controller = world.get_blueprint_library().find('controller.ai.walker')

    walker_object = random.choice(bp_walker)
    actor = world.try_spawn_actor(walker_object, transform)   
    world.wait_for_tick()  #to know if the player exist in the world 

    #Das brauchen wir nicht. Wir müssen es ohne der AI machen
    controller = world.spawn_actor(bp_walker_controller, carla.Transform(), actor) #carla.Transform() nicht so wichtig
    world.wait_for_tick()

    controller.start()
    controller.go_to_location(world.get_random_location_from_navigation)




In [ ]:
# Mit der Methode kann man Fußgänger beliebig in Abhängigkeit der Position des Autos platzieren 
def spawn_pedestrian_freePosition(self, distance_x, distance_y):
    transform = carla.Transform(self.get_transform().transform(carla.Location(x = distance_x, y = distance_y, z = 1)))
    spawn_pedestrian(transform)

# Die Methode ist wirklich zuständig für das Spawnen des Fußgängers. Man kann die Geschwindigkeit und die Richtung ändern.
def spawn_pedestrian(transform):
    bp_walker = world.get_blueprint_library().filter('walker.pedestrian.*')

    walker_object = random.choice(bp_walker)
    actor = world.try_spawn_actor(walker_object, transform)   
    world.wait_for_tick()  #to know if the player exist in the world 

    control = carla.WalkerControl()
    control.speed = 0.9
    control.direction.y = 0
    control.direction.x = 1
    control.direction.z = 0
    actor.apply_control(control)
    time.sleep(1)

    # boolean operation 1 for jump and 0 for not to jump
    control.jump = 0
    actor.apply_control(control)
    time.sleep(1)

    actor.apply_control(control)
    time.sleep(1)

    



In [ ]:
def spawn_pedestrian_freePosition(self, distance_x, distance_y, actor_speed, actor_direction_x, actor_direction_y):
    transform = carla.Transform(self.get_transform().transform(carla.Location(x = distance_x, y = distance_y, z = 1)))
    spawn_pedestrian(transform, actor_speed, actor_direction_x, actor_direction_y)


# Die Methode ist wirklich zuständig für das Spawnen des Fußgängers. Man kann die Geschwindigkeit und die Richtung ändern.
def spawn_pedestrian(transform, actor_speed, actor_direction_x, actor_direction_y):
    bp_walker = world.get_blueprint_library().filter('walker.pedestrian.*')

    walker_object = random.choice(bp_walker)
    actor = world.try_spawn_actor(walker_object, transform)   
    world.wait_for_tick()  #to know if the player exist in the world 

    control = carla.WalkerControl()
    control.speed = actor_speed
    control.direction.x = actor_direction_x
    control.direction.y = actor_direction_y
    control.direction.z = 0
    actor.apply_control(control)
    time.sleep(1)

    # boolean operation 1 for jump and 0 for not to jump
    control.jump = 0
    actor.apply_control(control)
    time.sleep(1)

    actor.apply_control(control)
    time.sleep(1)    

In [ ]:
#Hier werden die Bilder der Kamera in dem Ordner "outSzenario02" gespeichert
camera.listen(lambda image: image.save_to_disk('outSzenario02/%06d.png' % image.frame))

In [ ]:
# Set traffic in motion
for vehicle in world.get_actors().filter('*vehicle*'): 
    vehicle.set_autopilot(True) 

In [ ]:
#Hier spawnen wir die 4 Fußgänger
spawn_pedestrian_freePosition(ego_vehicle, 12, 4, 0.9, 0, 1)
spawn_pedestrian_freePosition(ego_vehicle, 15, -7,0.9, 0, -1)
time.sleep(0.5)
spawn_pedestrian_freePosition(ego_vehicle, 11, 4, 2, 0, 1)
time.sleep(0.4)
spawn_pedestrian_freePosition(ego_vehicle, 40, 4, 0.4, 0, 1)